In [78]:
#Get data for streets, driving, biking, walk paths
import osmnx as ox

#Library for analyziing complex networks and graphs
import networkx as nx

#Converts address to latitude and longitude
from geopy.geocoders import Nominatim

#Interactive Graphs
import folium

#Static Graphing
import matplotlib.pyplot as plt

#Display final map
from IPython.display import display

#Memory Efficent tools for working with iterations
import itertools

#Methods to get Geopy.geocoders to work
import ssl
import certifi

#Random Varibles
import random

In [80]:
# Create SSL context for secure geocoding
ssl_context = ssl.create_default_context(cafile=certifi.where())

# Initialize geolocator with SSL context
geolocator = Nominatim(user_agent='wichita_nav', ssl_context=ssl_context)

# Generate road network for Wichita, Kansas, USA
G_multi = ox.graph_from_place('Wichita, Kansas, USA', network_type='drive')

# Add speeds and travel times to edges
ox.add_edge_speeds(G_multi)
ox.add_edge_travel_times(G_multi)

# Convert MultiGraph to DiGraph, keeping only the fastest edge between nodes
def multigraph_to_digraph(G_multi):
    G_simple = nx.DiGraph()
    # Copy nodes with their attributes
    G_simple.add_nodes_from(G_multi.nodes(data=True))
    # Check if an edge already exists or if the new one is faster
    for u, v, data in G_multi.edges(data=True):
        if G_simple.has_edge(u, v):
            if data.get('travel_time', float('inf')) < G_simple[u][v].get('travel_time', float('inf')):
                G_simple[u][v].update(data)
        else:
            G_simple.add_edge(u, v, **data)
    return G_simple

# Convert MultiDiGraph to DiGraph
G = multigraph_to_digraph(G_multi)
# Copy the CRS attribute from the original graph
G.graph["crs"] = G_multi.graph.get("crs")

# Plot routes on Folium (interactive map)
def plot_route(G, route, route_map, color='blue'):
    route_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    folium.PolyLine(route_points, color=color, weight=4.5, opacity=0.7).add_to(route_map)

# Function to get nearest node from an address
def get_nearest_node(address): 
    location = geolocator.geocode(address, timeout=5)  # 5 sec timeout 
    if location:
        print(f"Geocoded Address: {address} -> ({location.latitude}, {location.longitude})")
        return ox.distance.nearest_nodes(G, X=location.longitude, Y=location.latitude)
    else: 
        print(f"Failed to geocode address: {address}")
        return None

# Example start and end addresses
start_address = "7331 Ayesbury Cir, Wichita KS"
end_address = "1220 N Tyler Rd, Wichita, KS"

start_node = get_nearest_node(start_address)
end_node = get_nearest_node(end_address)

# Helper function to compute total travel time of a route
def get_travel_time(G, route): 
    travel_time = 0
    for u, v in zip(route[:-1], route[1:]):
        travel_time += G[u][v]['travel_time']
    return travel_time

# Function for highway score: proportion of edges that are main roads
def highway_score(G, route): 
    main_road_types = ['motorway', 'trunk', 'primary', 'secondary']
    edges = list(zip(route, route[1:]))
    count = 0
    total = len(edges)
    for u, v in edges: 
        edge_data = G.get_edge_data(u, v)
        if edge_data: 
            highway = edge_data.get('highway', None)
            if isinstance(highway, list):
                if any(h in main_road_types for h in highway):
                    count += 1
            else:
                if highway in main_road_types:
                    count += 1
    return count / total if total > 0 else 0

# Define function to measure similarity between two routes
def route_similarity(route1, route2):
    set1, set2 = set(route1), set(route2)
    return len(set1.intersection(set2)) / min(len(set1), len(set2))

# Generate candidate routes using NetworkX's shortest_simple_paths with travel_time as weight
candidates = []
k = 50  # Maximum number of candidate routes to generate
gen = nx.shortest_simple_paths(G, start_node, end_node, weight='travel_time')

for i, route in enumerate(gen):
    if i >= k: 
        break
    travel_time = get_travel_time(G, route)
    score = highway_score(G, route)
    candidates.append({'route': route, 'travel_time': travel_time, 'highway_score': score})

# Sort candidates by travel time (ascending)
candidates = sorted(candidates, key=lambda x: x['travel_time'])
fastest_time = candidates[0]['travel_time'] if candidates else None

min_time_diff = 60  # Minimum time difference in seconds between selected routes
selected_routes = []

# Select the fastest route as route 1
if candidates:
    route1 = candidates[0]
    selected_routes.append(route1)

# Function to filter candidates that satisfy a minimum time difference and low similarity
def filter_candidates(candidates, last_selected, time_threshold, selected_routes):
    return [c for c in candidates 
            if c['travel_time'] >= last_selected['travel_time'] + time_threshold 
            and all(route_similarity(c['route'], sel['route']) < 0.20 for sel in selected_routes)]

# Attempt to select route 2: choose the candidate with the highest highway score that meets the time difference constraint from route1
if len(selected_routes) == 1:
    candidates_route2 = filter_candidates(candidates, selected_routes[0], min_time_diff, selected_routes)
    if candidates_route2:
        route2 = max(candidates_route2, key=lambda x: x['highway_score'])
        selected_routes.append(route2)

# Attempt to select route 3: choose the candidate with the highest highway score that is at least 60 seconds slower than route2
if len(selected_routes) == 2:
    candidates_route3 = filter_candidates(candidates, selected_routes[1], min_time_diff, selected_routes)
    if candidates_route3:
        route3 = max(candidates_route3, key=lambda x: x['highway_score'])
        selected_routes.append(route3)

# Fallback: if less than 3 routes are found, use the top 3 candidates by travel time
if len(selected_routes) < 3:
    print('Fewer than 3 distinct routes meeting the time gap criteria were found. Falling back to the top 3 by travel time.')
    selected_routes = candidates[:3]

# Debug printout for candidate routes
for idx, candidate in enumerate(selected_routes):
    print(f"Route {idx+1}: Travel Time = {candidate['travel_time']:.1f} sec, Highway Score = {candidate['highway_score']:.2f}")

# Initialize Folium map centered on Wichita
location_wichita = geolocator.geocode("Wichita, Kansas, USA")
route_map = folium.Map(location=[location_wichita.latitude, location_wichita.longitude], zoom_start=12)

# Define colors for each route
colors = ['blue', 'red', 'green']

# Plot selected routes on the map
for i, candidate in enumerate(selected_routes):
    plot_route(G, candidate['route'], route_map, color=colors[i])

# Display the map (in Jupyter, the map will render in the output cell)
route_map



Geocoded Address: 7331 Ayesbury Cir, Wichita KS -> (37.727137306122444, -97.25127428571429)
Geocoded Address: 1220 N Tyler Rd, Wichita, KS -> (37.70351867346939, -97.44421955102041)
Fewer than 3 distinct routes meeting the time gap criteria were found. Falling back to the top 3 by travel time.
Route 1: Travel Time = 1216.6 sec, Highway Score = 0.87
Route 2: Travel Time = 1217.1 sec, Highway Score = 0.81
Route 3: Travel Time = 1218.2 sec, Highway Score = 0.81
